# 25. テンプレートの抽出

記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [ ]:
# 問題25: テンプレートの抽出

import os
import re

# イギリスの記事ファイルのパス
data_dir = "../data"
uk_article_file = os.path.join(data_dir, "uk_article.txt")

# 基礎情報テンプレートのフィールド名と値を抽出する関数
def extract_basic_info_template(text):
    """記事から基礎情報テンプレートのフィールド名と値を抽出する関数
    
    Args:
        text: 記事のテキスト
        
    Returns:
        フィールド名と値の辞書
    """
    # 基礎情報テンプレートの抽出
    template_pattern = r"\{\{基礎情報[^|]*?\|([\s\S]*?)\}\}"
    template_match = re.search(template_pattern, text)
    
    if not template_match:
        return {}
    
    template_content = template_match.group(1)
    
    # フィールドの抽出
    field_pattern = r"\|\s*([^|\}]+?)\s*=\s*([^|]*?)(?:\||\}\}$)"
    fields = re.findall(field_pattern, template_content + "}}")
    
    # 辞書に格納
    basic_info = {}
    for name, value in fields:
        # 前後の空白を削除
        name = name.strip()
        value = value.strip()
        basic_info[name] = value
    
    return basic_info

# メイン処理
try:
    # イギリスの記事を読み込む
    if os.path.exists(uk_article_file):
        with open(uk_article_file, "r", encoding="utf-8") as f:
            uk_article = f.read()
        
        # 基礎情報テンプレートのフィールド名と値を抽出
        basic_info = extract_basic_info_template(uk_article)
        
        # 結果を表示
        print(f"基礎情報テンプレートのフィールド数: {len(basic_info)}\n")
        print("基礎情報テンプレートの内容:")
        for i, (name, value) in enumerate(basic_info.items()):
            # 値が長い場合は省略
            if len(value) > 100:
                value = value[:100] + "..."
            print(f"{i+1}. {name}: {value}")
    else:
        print(f"イギリスの記事ファイルが見つかりません: {uk_article_file}")
        print("問題20を先に実行して、イギリスの記事を抽出してください。")
        
except Exception as e:
    print(f"エラーが発生しました: {e}")

In [ ]:
# 改良版: 複数行にまたがるフィールド値に対応

def extract_basic_info_template_improved(text):
    """複数行にまたがるフィールド値に対応した基礎情報テンプレート抽出関数
    
    Args:
        text: 記事のテキスト
        
    Returns:
        フィールド名と値の辞書
    """
    # 基礎情報テンプレートの抽出
    template_pattern = r"\{\{基礎情報[^|]*?\|([\s\S]*?)\}\}"
    template_match = re.search(template_pattern, text)
    
    if not template_match:
        return {}
    
    template_content = template_match.group(1)
    
    # フィールドの抽出（行ごとに処理）
    basic_info = {}
    field_name = None
    field_value = []
    
    for line in template_content.split("\n"):
        # 新しいフィールドの開始
        field_match = re.match(r"\|\s*([^=]+?)\s*=\s*(.*)", line)
        if field_match:
            # 前のフィールドがあれば保存
            if field_name is not None:
                basic_info[field_name] = "\n".join(field_value).strip()
            
            # 新しいフィールドの開始
            field_name = field_match.group(1).strip()
            field_value = [field_match.group(2).strip()]
        else:
            # 現在のフィールドの値の続き
            if field_name is not None:
                field_value.append(line.strip())
    
    # 最後のフィールドを保存
    if field_name is not None:
        basic_info[field_name] = "\n".join(field_value).strip()
    
    return basic_info

# イギリスの記事ファイルが存在する場合のみ実行
if os.path.exists(uk_article_file):
    with open(uk_article_file, "r", encoding="utf-8") as f:
        uk_article = f.read()
    
    # 改良版の関数を使用して基礎情報テンプレートを抽出
    basic_info_improved = extract_basic_info_template_improved(uk_article)
    
    # 結果を表示
    print(f"\n改良版: 基礎情報テンプレートのフィールド数: {len(basic_info_improved)}\n")
    
    # 2つの方法の結果を比較
    print("2つの方法の比較:")
    print(f"方法1のフィールド数: {len(basic_info)}")
    print(f"方法2のフィールド数: {len(basic_info_improved)}")
    
    # 方法1にあって方法2にないフィールド
    only_in_method1 = set(basic_info.keys()) - set(basic_info_improved.keys())
    if only_in_method1:
        print(f"\n方法1のみに含まれるフィールド: {only_in_method1}")
    
    # 方法2にあって方法1にないフィールド
    only_in_method2 = set(basic_info_improved.keys()) - set(basic_info.keys())
    if only_in_method2:
        print(f"\n方法2のみに含まれるフィールド: {only_in_method2}")

In [ ]:
# 基礎情報テンプレートの特定のフィールドを表示

def display_specific_fields(basic_info, fields_to_display):
    """基礎情報テンプレートの特定のフィールドを表示する関数
    
    Args:
        basic_info: 基礎情報テンプレートの辞書
        fields_to_display: 表示するフィールド名のリスト
    """
    print("\n特定のフィールドの内容:")
    for field in fields_to_display:
        if field in basic_info:
            value = basic_info[field]
            # 値が長い場合は省略
            if len(value) > 100:
                value = value[:100] + "..."
            print(f"{field}: {value}")
        else:
            print(f"{field}: フィールドが見つかりません")

# イギリスの記事ファイルが存在する場合のみ実行
if os.path.exists(uk_article_file):
    # 表示する特定のフィールド
    fields_to_display = ["正式名称", "国旗画像", "国章画像", "位置画像", "公用語", "首都", "最大都市"]
    
    # 特定のフィールドを表示
    display_specific_fields(basic_info_improved, fields_to_display)

## 解説

この問題では、記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し、辞書オブジェクトとして格納する方法を学びます。

### 基礎情報テンプレートの形式

Wikipediaの記事では、基礎情報テンプレートは以下の形式で表現されています：

```
{{基礎情報 国
|略名 = イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}
...
}}
```

### 抽出の手順

1. **テンプレート全体の抽出**：
   - 正規表現を使用して、`{{基礎情報 ...}}` の形式のテンプレート全体を抽出します。

2. **フィールドの抽出**：
   - テンプレート内の各フィールドを抽出します。
   - フィールドは `|フィールド名 = 値` の形式で表現されています。

### 2つの実装方法

1. **正規表現のみを使用する方法**：
   - 正規表現を使用して、テンプレート全体とフィールドを一度に抽出します。
   - 複数行にまたがるフィールド値の処理が難しい場合があります。

2. **行ごとに処理する方法**：
   - テンプレート全体を抽出した後、行ごとに処理します。
   - 新しいフィールドの開始を検出し、それまでの値を保存します。
   - 複数行にまたがるフィールド値も正しく処理できます。

### 注意点

- テンプレート内のフィールド値には、WikipediaのマークアップやHTMLタグが含まれている場合があります。
- フィールド値が複数行にまたがる場合、単純な正規表現では正確に抽出できないことがあります。
- テンプレートの構造が複雑な場合（入れ子になったテンプレートなど）、より高度な解析が必要になることがあります。
- 実際のWikipediaの記事では、テンプレートの形式が統一されていない場合があります。